In [1]:
import datetime
import numpy as np
import matplotlib.pyplot as plt
from math import factorial, fabs

np.set_printoptions(precision=6)

In [2]:
def get_stationary_vector(omega, Theta):
#     while any(omega != omega.dot(Theta)):
#         omega = omega.dot(Theta)
    for _ in range(100000):
        omega = omega.dot(Theta)
    return omega  

In [3]:
L = 6
R = 4
Q = 6
lambda0 = .3
k = [1, 1, 1, 1, 1, 1]
mu = [.5, .5, .5, .5, .5, .5]
Theta = np.array([
    [.0, .5, .0, .0, .5, .0, .0],
    [.0, .0, .6, .4, .0, .0, .0],
    [.0, .2, .0, .8, .0, .0, .0],
    [.3, .0, .0, .0, .7, .0, .0],
    [.0, .0, .0, .0, .0, .5, .5],
    [.3, .0, .0, .0, .0, .0, .7],
    [.8, .0, .0, .0, .2, .0, .0]
])

omega = np.array([1 / (L + 1) for _ in range(L + 1)])
omega = get_stationary_vector(omega, Theta)
print('Стационарное решение: ')
print(omega, '\n', sum(omega))

Стационарное решение: 
[0.207653 0.117985 0.070791 0.103827 0.212657 0.106329 0.180759] 
 1.0000000000000002


In [4]:
# A: mu - произвольный, подобрать Theta -> max(lambdas)
def get_lambdas(omega, theta, lambda0):
    omega = get_stationary_vector(omega, theta)
    lambdas = [0] * len(omega)
    for i in range(1, len(omega)):
        lambdas[i - 1] = lambda0 * (omega[i] / omega[0])
    return lambdas

lambdas = None
sum_lambdas = 0

print(datetime.datetime.now())

for s0 in np.arange(.1, .9, .15):
    for s1 in np.arange(.1, .9, .15):
        for s2 in np.arange(.1, .9, .15):
            for s3 in np.arange(.1, .9, .15):
                for s4 in np.arange(.1, .9, .15):
                    for s5 in np.arange(.1, .9, .15):
                        for s6 in np.arange(.1, .9, .15):
                            Theta[0][1], Theta[0][4] = s0, 1 - s0
                            Theta[1][2], Theta[1][3] = s1, 1 - s1
                            Theta[2][1], Theta[2][3] = s2, 1 - s2
                            Theta[3][0], Theta[3][4] = s3, 1 - s3
                            Theta[4][5], Theta[4][6] = s4, 1 - s4
                            Theta[5][0], Theta[5][6] = s5, 1 - s5
                            Theta[6][0], Theta[6][4] = s6, 1 - s6
                            lambdas = get_lambdas(omega, Theta, lambda0)
                            if sum_lambdas <= sum(lambdas):
                                sum_lambdas = sum(lambdas)
                                max_lambdas = lambdas
                                print(sum(lambdas))

print(f'\nПропускная способность сети {sum_lambdas} максимальная при:')
print('lambdas: ', max_lambdas)
print('Theta:\n', Theta)
print(datetime.datetime.now())

2020-04-17 17:41:26.727280
5.758103975535169
5.758616796047988
5.759145642201834
5.759691277122469
5.760254513169576
5.760836215972326
5.763232180663374
5.764770642201833
5.7664373088685
5.768248903071399
5.770225187656379
5.772389689820882
5.774770642201835
5.778616796047988
5.783103975535168
5.7884070058382
5.791437308868501
5.800380398299396
5.812094585863806
5.824770642201834
5.850696568127761
5.861807679238872
5.924770642201834
5.93348623853211
5.998301053346925
6.026078831124703
6.18348623853211
6.190349983010533
6.442201834862384
6.700917431192659
6.959633027522935
7.2183486238532115

Пропускная способность сети 7.2183486238532115 максимальная при:
lambdas:  [0.9189189189189186, 0.7810810810810812, 0.255, 2.518348623853211, 0.25183486238532116, 2.4931651376146795, 0]
Theta:
 [[0.   0.85 0.   0.   0.15 0.   0.  ]
 [0.   0.   0.85 0.15 0.   0.   0.  ]
 [0.   0.85 0.   0.15 0.   0.   0.  ]
 [0.85 0.   0.   0.   0.15 0.   0.  ]
 [0.   0.   0.   0.   0.   0.85 0.15]
 [0.85 0.   0.   

In [5]:
# B: Theta - произвольная, подобрать mu -> sum(mu) -> min при tau = Q
def get_tau(L, k, mu, lambda0, Theta, omega):
        lambdas = get_lambdas(omega, Theta, lambda0) 
        
        psy = [0] * L
        for i in range(len(psy)):
            psy[i] = lambdas[i] / (k[i] * mu[i])
        
        P0 = [0] * L
        bi = [0] * L
        h = [0] * L
        n = [0] * L
        u = [0] * L
        tau = 0

        for i in range(len(P0)):
            summa = 0
            for j in range(k[i]):
                summa += (k[i] * psy[i])**j / factorial(j)
            P0[i] = ((((k[i] * psy[i])**k[i]) / 
                          (factorial(k[i]) * (1 - psy[i])) ) + summa)**(-1)
            bi[i] = P0[i] * (((k[i]**k[i]) * (psy[i]**(k[i] + 1))) / 
                                (factorial(k[i]) * ((1 - psy[i])**2)))
            h[i] = psy[i] * k[i]
            n[i] = bi[i] + h[i]
            u[i] = n[i] / lambdas[i]
            tau += lambdas[i] * u[i]

        tau = (1 / lambda0) * tau
        return tau

Theta = np.array([
    [.0, .5, .0, .0, .5, .0, .0],
    [.0, .0, .6, .4, .0, .0, .0],
    [.0, .2, .0, .8, .0, .0, .0],
    [.3, .0, .0, .0, .7, .0, .0],
    [.0, .0, .0, .0, .0, .5, .5],
    [.3, .0, .0, .0, .0, .0, .7],
    [.8, .0, .0, .0, .2, .0, .0]
])
    
eps = 0.1
# шаг
delta = 0.0001
# начальный вектор
mu = [.1, .1, .1, .1, .1, .1]
i = 0
tau = get_tau(L, k, mu, lambda0, Theta, omega)
while fabs(tau - Q) > eps:
    mu[i % 6] += delta
    i += 1
    tau = get_tau(L, k, mu, lambda0, Theta, omega)
#     print('mu: ', mu)
#     print('tau:', tau)


print(f'\nСумма компонентов вектора mu минимальна и равна {sum(mu)}')
print(f'Время реакции равно {Q} плюс-минус {eps}')
print('mu: ', mu)
print('tau:', tau)
print('Theta:\n', Theta)


Сумма компонентов вектора mu минимальна и равна 0.6543000000000017
Время реакции равно 6 плюс-минус 0.1
mu:  [0.10910000000000027, 0.10910000000000027, 0.10910000000000027, 0.10900000000000026, 0.10900000000000026, 0.10900000000000026]
tau: 6.08302754351935
Theta:
 [[0.  0.5 0.  0.  0.5 0.  0. ]
 [0.  0.  0.6 0.4 0.  0.  0. ]
 [0.  0.2 0.  0.8 0.  0.  0. ]
 [0.3 0.  0.  0.  0.7 0.  0. ]
 [0.  0.  0.  0.  0.  0.5 0.5]
 [0.3 0.  0.  0.  0.  0.  0.7]
 [0.8 0.  0.  0.  0.2 0.  0. ]]
